In [1]:
import numpy as np
from classifiers import *
from timeit import default_timer as timer
from cross_validation import *
import data_storage

In [2]:
%load_ext autoreload
%autoreload 2

# Load Dataset

In [9]:
def log(string, style='a'):
    print(string)
    global dataset_name
    file_name = dataset_name + '_log.txt'
    with open(file_name, style) as file:
        file.write(str(string) + '\n')
        

k = 5
data_storage.init(k)
classes = np.arange(k)

dataset_name = 'SimplexTesting'

surrogate = 'AT'


num_quantiles = 1 # int(round(5 * np.log10(k)))
gammas= [1.0 * i / (num_quantiles + 1) for i in range(1, num_quantiles + 1)]
log(gammas)

kernel_type = 'linear'
loss_function = 'logistic'

opt_type = 'SciPy'
opt_params = {'learning_rate': 1e-6, 'momentum_gamma': 0.9, 'batch_size': 100}


log('Cross Validating parameters')
cv_dir_name_base = 'cv_' + dataset_name
kernel_params = [1e-6]
reg_params = [1e-4, 1]

best_parameters = cross_validate_large_kernel_grid(surrogate, kernel_type, kernel_params, reg_params, data_storage.X_train, data_storage.y_train, classes,
                                             gammas, loss_function, opt_type, opt_params, cv_dir_name_base)

log(best_parameters)
with open(dataset_name + '_cv_parameter_dict.csv', 'w') as csv_file:
    writer = csv.writer(csv_file)
    for key, value in best_parameters.items():
        writer.writerow([key, value])

(gammas, alphas, kernel_params) = get_params(best_parameters, surrogate)

log(gammas)
log(alphas)
log(kernel_params)


[0.5]
Cross Validating parameters


/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example using ravel().
  y = column_or_1d(y, warn=True)
/anaconda3/lib/python3.7/site-packages/sklearn/utils/validation.py:578: DataConversionWarning: A column-vector y was passed when a 1d array was expected. Please change the shape of y to (n_samples, ), for example usin

{0.5: {'AT': <DictProxy object, typeid 'dict' at 0x1a18770860>, 'IT': <DictProxy object, typeid 'dict' at 0x1a18770f60>}}
0.525
0.425
0.525
0.425
0.425
0.525
0.525
0.4
0.475
0.475
{'0.0001_1e-06': 2.375, '1_1e-06': 2.35}
1_1e-06
{'AT': {0.5: {'reg': 1.0, 'kernel': 1e-06}}, 'IT': {0.5: {'reg': 0.0001, 'kernel': 1e-06}}}
[0.5]
[1.0]
[1e-06]


In [ ]:
train_path = 'datasets/mnist_train.csv'
test_path = 'datasets/mnist_test.csv'
y_col = 0

In [ ]:
def get_features_and_labels(dataset, y_col):
    y = dataset[:, y_col:y_col+1]
    selector = [col for col in range(dataset.shape[1]) if col != y_col]
    X = dataset[:, selector]
    return X, y

In [ ]:
## Import data and get sets
train_set = np.genfromtxt(train_path, delimiter=',')
test_set = np.genfromtxt(test_path, delimiter=',')

train_X, train_y = get_features_and_labels(train_set, y_col)
test_X, test_y = get_features_and_labels(test_set, y_col)

In [ ]:
# Optional: Truncate datasets/mess around with them as needed
n = 1000
train_X, train_y = train_X[0:n, :], train_y[0:n, :]
test_X, test_y = test_X[0:n, :], test_y[0:n, :]

#Normalize Data
train_X, train_y = train_X - np.mean(train_X) , train_y
test_X, test_y = test_X - np.mean(test_X) , test_y

In [ ]:
n = test_y.size
train_y = train_y.astype(int)
test_y = test_y.astype(int)

# Run Quantile Regression 

In [ ]:
def train_classifiers(X_train, y_train, X_test, y_test, a, reg_param=1,
                      kernel='linear', kernel_param=1, loss_function='logistic',
                      opt_type='SGD', opt_params={'learning_rate': 1}):
    results = {}
    
    base = opt_params['plot_file']
    opt_params['plot_file'] = 'IT' + base
    start = timer()
    clf5 = QuantileIT(gamma=a, alpha=reg_param, kernel_type=kernel, opt_type=opt_type, opt_params=opt_params,
                              kernel_param=kernel_param, loss_function=loss_function)
    clf5.fit(X_train, y_train)
    preds_5 = clf5.predict(X_test)
    end = timer()
    
    abs_loss = weighted_absolute_loss(preds_5, y_test, a)
    print('Weighted Absolute Loss of QuantileIT, gamma=' + str(a) + ' %s' %
          abs_loss)
    
    preds_5_in = clf5.predict(X_train)
    abs_loss_in = weighted_absolute_loss(preds_5_in, y_train, a)
    print('In Sample Weighted Absolute Loss of QuantileIT, gamma=' + str(a) + ' %s' %
          abs_loss_in)
    print('Time Elapsed: ' + str(end - start))
    
    results[clf5] = {'AbsLoss': abs_loss, 'Preds': preds_5}
    
    print(' ')

    opt_params['plot_file'] = 'AT' + base
    start = timer()
    clf6 = QuantileAT(gamma=a, alpha=reg_param, kernel_type=kernel, opt_type=opt_type, opt_params=opt_params,
                              kernel_param=kernel_param, loss_function=loss_function)
    clf6.fit(X_train, y_train)
    preds_6 = clf6.predict(X_test)
    end = timer()
    abs_loss = weighted_absolute_loss(preds_6, y_test, a)
    print('Weighted Absolute Loss of QuantileAT, gamma=' + str(a) + ' %s' %
          abs_loss)
    
    preds_6_in = clf6.predict(X_train)
    print('In Sample Weighted Absolute Loss of QuantileAT, gamma=' + str(a) + ' %s' %
          abs_loss_in)
    print('Time Elapsed: ' + str(end - start))
    
    results[clf6] = {'AbsLoss': abs_loss, 'Preds': preds_6}
    return results

In [ ]:
reg = 10
kern = 'linear'
kern_param = 1
lf = 'hinge'

parms = {'learning_rate': 0.00000001, 'momentum_gamma': 0.9,
         'batch_size': 500, 'plot_file': 'test.png'}

results = train_classifiers(train_X, train_y, test_X, test_y, 0.3, reg_param=reg,
                            kernel=kern, kernel_param=kern_param, loss_function=lf,
                            opt_type='SGD', opt_params=parms)

# Run Multiclass Test
Learning $3 \lg k$ quantiles

In [ ]:
surrogate='AT'
num_quantiles = 5
gammas= [i / (num_quantiles + 1) for i in range(1, num_quantiles + 1)]
alphas=[1000, 100, 10, 10, 1000]
kernel_type='linear'
kernel_param = 10
loss_function='hinge'
opt_type = 'Momentum'
opt_params={'learning_rate': 1e-8, 'momentum_gamma': 0.9, 'batch_size': 100}

In [ ]:
clf = QuantileMulticlass(surrogate=surrogate, gammas=gammas, alphas=alphas, 
                         kernel_type=kernel_type, kernel_param=kernel_param, loss_function=loss_function, 
                         opt_type=opt_type, opt_params=opt_params)

In [ ]:
clf.fit(train_X, train_y)

In [ ]:
preds = clf.predict_score(test_X)

In [ ]:
preds_vote = clf.predict(test_X)

In [ ]:
print(weighted_absolute_loss(preds, test_y, 0.5))
print(weighted_absolute_loss(preds_vote, test_y, 0.5))

In [ ]:
print(zo_loss(preds, test_y))
print(zo_loss(preds_vote, test_y))

In [ ]:
np.concatenate((preds, test_y), axis=1)

In [ ]:
surrogate='IT'
num_quantiles = 5
gammas= [i / (num_quantiles + 1) for i in range(1, num_quantiles + 1)]
alphas=[1, 1000, 1000, 1, 10]
kernel_type='linear'
kernel_param = 10
loss_function='hinge'
opt_type = 'Momentum'
opt_params={'learning_rate': 1e-8, 'momentum_gamma': 0.9, 'batch_size': 100}

clf = QuantileMulticlass(surrogate=surrogate, gammas=gammas, alphas=alphas, 
                         kernel_type=kernel_type, kernel_param=kernel_param, loss_function=loss_function, 
                         opt_type=opt_type, opt_params=opt_params)

clf.fit(train_X, train_y)

preds = clf.predict_score(test_X)

preds_vote = clf.predict(test_X)

print(weighted_absolute_loss(preds, test_y, 0.5))
print(weighted_absolute_loss(preds_vote, test_y, 0.5))

print(zo_loss(preds, test_y))
print(zo_loss(preds_vote, test_y))

# Run Multiclass Algorithms (for Comparison)

In [ ]:
# Standard Logisitic regression
from sklearn.linear_model import LogisticRegression
clf_log = LogisticRegression(random_state=0, solver='lbfgs', multi_class='multinomial', fit_intercept=False).fit(train_X, train_y)
metrics.zero_one_loss(clf_log.predict(test_X), test_y)

In [ ]:
weighted_absolute_loss(clf_log.predict(test_X), test_y, 0.5)